In [ ]:
# JuliaSystems toolboxes
#Pkg.clone("https://github.com/JuliaSystems/SystemsBase.jl.git")
#Pkg.clone("https://github.com/JuliaSystems/ControlToolbox.jl.git")
#Pkg.clone("https://github.com/neveritt/IdentificationToolbox.jl.git")
Pkg.checkout("IdentificationToolbox","WIP-MIMO")
Pkg.checkout("ControlToolbox","development")

# Plotting functionality
Pkg.add("Plots")
Pkg.add("GR")

Pkg.update()

In [2]:
using IdentificationToolbox
using ControlToolbox


Use "local opt::Core.getfield(Optim,:OptimizationResults)" instead.

Use "(::Type{DSisoRational})(...)" instead.

Use "local Ts::Float64" instead.

Use "local Ts::Float64" instead.

Use "(::Type{DSisoZpk})(...)" instead.

Use "local Ts::Float64" instead.

Use "local Ts::Float64" instead.

Use "(::Type{DMimo})(...)" instead.

Use "(::Type{CSisoRational})(...)" instead.

Use "(::Type{CSisoZpk})(...)" instead.

Use "(::Type{CMimo})(...)" instead.

Use "(::Type{CSisoSs})(...)" instead.

Use "(::Type{DSisoSs})(...)" instead.

Use "(::Type{CMimoSs})(...)" instead.

Use "(::Type{DMimoSs})(...)" instead.


LoadError: LoadError: LoadError: UndefVarError: StateSpace not defined
while loading /home/neveritt/.julia/v0.5/ControlToolbox/src/c2d.jl, in expression starting on line 13
while loading /home/neveritt/.julia/v0.5/ControlToolbox/src/ControlToolbox.jl, in expression starting on line 33

In [3]:
data, header = readcsv("collected-data.csv", header=true)

(
[0.0 -0.206242 0.644553; 0.1001 -1.17932 0.586008; … ; 99.8999 -1.22364 1.99638; 100.0 -4.12476 2.20096],

AbstractString["t" "u" "y"])

In [4]:
u = data[:,2].'; y = data[:,3].';
zdata = iddata(y,u)

Discrete-time data set with 1000 samples.
Sampling time: 1.0 seconds


In [11]:
model = OE(2,2,[1]);
pem(zdata, model, zeros(6))

LoadError: DimensionMismatch("matrix A has dimensions (1,1000), matrix B has dimensions (1,1)")

In [ ]:
Pkg.rm("IdentificationToolbox")

### Aircraft Pitch: System Modeling

Now we will simulate the time-response of the aircraft pitch model given below:

\begin{align*}
  \begin{bmatrix}
    \dot{\alpha} \\ \dot{q} \\ \dot{\theta}
  \end{bmatrix} & = \begin{bmatrix}
    -0.313  & 56.7   & 0 \\
    -0.0139 & -0.426 & 0 \\
     0      & 56.7   & 0
  \end{bmatrix} \begin{bmatrix}
    \alpha \\ q \\ \theta
  \end{bmatrix} + \begin{bmatrix}
    0.232 \\ 0.0203 \\ 0
  \end{bmatrix} \delta \\
  y & = \begin{bmatrix}
    0 & 0 & 1
  \end{bmatrix} \begin{bmatrix}
    \alpha \\ q \\ \theta
  \end{bmatrix}
\end{align*}

In [ ]:
A = [-0.313   56.7  0;
     -0.0139 -0.426 0;
      0       56.7  0]
B = [0.232 ;
     0.0203;
     0     ]
C = [0 0 1]
D = [0]

Pₚ = ss(A,B,C,D)

### Analysis

Now let's see how the uncompensated open-loop system performs. Specifically, we will use the command step to analyze the open-loop step response where we have scaled the input to represent an elevator angle input ($\delta$) of 0.2 radians (11 degrees)

t = collect(0:0.01:10)
step(0.2*Pₚ,t);

The open loop response is clearly unstable. Stability of a system can be determined by examining the poles of the transfer function where the poles can be identified using the command poles as shown below.

In [ ]:
poles(Pₚ)

#### Closed-loop response

In order to stabilize this system and eventually meet our given design requirements, we will add a feedback controller.

In [ ]:
sys_cl = feedback(Pₚ,1)
poles(sys_cl)

In [ ]:
zeros(sys_cl)

### PID design

### root locus design

### Controllability

In order to apply our state-space controller design techniques we need to first verify an important property, controllability. The controllability property is necessary to guarantee that we have the authority to drive the state of the system anywhere we like. This corresponds to the ability to place the closed-loop poles of the system anywhere in the complex s-plane.

The controllability Gramian is a Gramian used to determine whether or not a linear system is controllable. The controllability Gramian  is given as the 
unique solution of the Lyapunov equation

\begin{align*}
  AW_{c}+W_{c}A^{T} & = -BB^{T}
\end{align*}

is positive definite if and only if the pair $(A, B)$ is controllable.

In [ ]:
co = gram(Pₚ, :c) # controllability grammian

eig(co) # check positive definiteness

### Pole placement

Referring back to the state-space equations at the top of the page, we see that substituting the state-feedback law $\delta$ = $\theta_r$ - K x for $\delta$ leads to the following.

\begin{align*}
  \dot{{\bf x}} & = (A - BK){\bf x} + B\theta_{r} \\
  \theta & = C{\bf x}
\end{align*}

Based on the above, matrix $A - BK$ determines the closed-loop dynamics of our system.

We know from the above that we can place the closed-loop poles of the system anywhere we would like. The question then that is left is, where should we place them? If we have a standard first- or second-order system, we then have relationships that directly relate pole locations to characteristics of the step response and can use these relations to place the poles in order to meet our given requirements. This process becomes more difficult if we have a higher-order system or zeros. With a higher-order system, one approach is to place the higher-order poles 5-10 times farther to the left in the complex plane than the dominant poles, thereby leading them to have negligible contribution to the transient response. The effect of zeros is more difficult to address using a pole-placement approach to control. Another limitation of this pole-placement approach is that it doesn't explicitly take into account other factors like the amount of required control effort.

In [ ]:
p₁ = [-10, -0.8+0.5im, -0.8-0.5im] # desired poles
p₂ = [-1, -1, -1]
p₃ = [-0.6, -0.6+0.6im, -0.6-0.6im]
P  = [p₁, p₂, p₃]

for p in P
    K = place(Pₚ.A, Pₚ.B, p)
    
    sys_cl = ss(A-K*B, B, C, D)
    step(sys_cl)
end

### Linear quadratic regulation

We will use a technique called the Linear Quadratic Regulator (LQR) method to generate the "best" gain matrix K, without explicitly choosing to place the closed-loop poles in particular locations. This type of control technique optimally balances the system error and the control effort based on a cost that the designer specifies that defines the relative importance of minimizing errors and minimimizing control effort. In the case of the regulator problem, it is assumed that the reference is zero. Therefore, in this case the magnitude of the error is equal to the magnitude of the state. To use this LQR method, we need to define two parameters: the state-cost weighted matrix (Q) and the control weighted matrix (R). For simplicity, we will choose the control weighted matrix equal to 1 (R=1), and the state-cost matrix (Q) equal to pC'C. Employing the vector C from the output equation means that we will only consider those states in the output in defining our cost. In this case,  $\theta$ is the only state variable in the output. The weighting factor (p) will be varied in order to modify the step response. In this case, R is a scalar since we have a single input system.

In [ ]:
p = 2
Q = p*C'*C
R = 1
K = lqr(A,B,Q,R)

Note the structure of the weighting matrix Q and the resulting gain matrix K. Referring to the closed-loop state equations given above assuming a control law with non-zero reference, $\delta = \theta_{r}$ - K x $. Note that the response is scaled to model the fact that the pitch angle reference is a 0.2 radian (11 degree) step. The step response shown below should then be generated.

In [ ]:
sys_cl = ss(A-B*K, B, C, D)
step(0.2*sys_cl)

### adding precompensation

We try to make the response faster by penalizing the system error more by increasing $p$. Unlike other design methods, the full-state feedback system does not compare the output to the reference; instead, it compares all states multiplied by the control matrix $Kx$ to the reference. Thus, we should not expect the output to equal the commanded reference. To obtain the desired output, we can scale the reference input so that the output does equal the reference in steady state. This can be done by introducing a precompensator scaling factor called $\bar{N}$.

In [ ]:
p = 50;
Q = p*C'*C;
R = 1;
K = lqr(A,B,Q,R);
sys_cl = ss(A-B*K, B, C, D)

N̄ = 1/sys_cl(0)

sys_cl = ss(A-B*K,B*N̄,C,D)
step(0.2*sys_cl)

### Discrete state-space

The first step in the design of a digital control system is to generate a sampled-data model of the plant. 

In choosing a sample time, note that it is desired that the sampling frequency be fast compared to the dynamics of the system in order that the sampled output of the system captures the system's full behavior, that is, so that significant inter-sample behavior isn't missed. One measure of a system's "speed" is its closed-loop bandwidth. A good rule of thumb is that the sampling time be smaller than 1/30th of the closed-loop bandwidth frequency.
From the closed-loop Bode plot, the bandwidth frequency can be determined to be approximately 2 rad/sec (0.32 Hz). You may verify this yourself. Thus, to be sure we have a small enough sampling time, we will use a sampling time of 1/100 sec/sample. 

In [ ]:
Ts = 0.01
sys_d, x0map = c2d(sys_ss, Ts, Discretization.Bilinear())

For a system in state space form, returns the discretized system as well as a
matrix $x0map$ that transforms the initial conditions to the discrete domain by
x0_discrete = x0map*[x0, u0].